# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys # usar o teclado
from selenium.webdriver.common.by import By # localizar os itens no navegador

#criar navegador
navegador = webdriver.Chrome()

# entrar no google e pesquisar por cotação do dolar
navegador.get("https://www.google.com/")

# pegar cotação do dolar
navegador.find_element(By.XPATH,
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dolar")

navegador.find_element(By.XPATH,
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print("Dólar:", cotacao_dolar)

# entrar no google e pesquisar cotação do euro
navegador.get("https://www.google.com/")

# pegar cotação do euro
navegador.find_element(By.XPATH,
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")

navegador.find_element(By.XPATH,
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print("Euro:", cotacao_euro)

# entrar no https://www.melhorcambio.com/ouro-hoje
navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute('value')

cotacao_ouro = cotacao_ouro.replace(",", ".")
# pegar cotação do ouro
print("Ouro:", cotacao_ouro)

navegador.quit()



### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [ ]:
# importar a base de dados dos produtos
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

- Atualizando os preços e o cálculo do Preço Final

In [ ]:
# atualizar as cotações na base de dados

# cotação do dolar
tabela.loc[tabela["Moeda"]=="Dólar","Cotação"] = float(cotacao_dolar)

#cotação do euro
tabela.loc[tabela["Moeda"]=="Euro","Cotação"] = float(cotacao_euro)

#cotação do ouro
tabela.loc[tabela["Moeda"]=="Ouro","Cotação"] = float(cotacao_ouro)

#preço de compra = preço original * cotação
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# preço de compra = preço original * margem
tabela["Preço de Vendas"] = tabela["Preço Original"] * tabela ["Margem"]

display(tabela)

In [ ]:
### Agora vamos exportar a nova base de preços atualizada
tabela.to_excel("ProdutosNovo.xlsx", index= False)
